# TetCTF 2020 - (Crypto) 2019rearrange

Challenge's code:

```python
import random
from Crypto.Util.number import getPrime


def encrypt(m, n):
    digits = [2, 0, 1, 9]
    random.shuffle(digits)
    e = sum([d * 10 ** (3 - i) for i, d in enumerate(digits)])
    noise = random.randint(0, n - 1)
    return e, noise, pow(m + noise, e, n)


if __name__ == '__main__':
    p = getPrime(512)
    q = getPrime(512)
    n = p * q
    print("n =", n)

    m = int.from_bytes(open('flag.txt', 'rb').read(), 'big')

    # can you recover m from 2 ciphertexts?
    print('e1, noise1, c1 =', encrypt(m, n))
    print('e2, noise2, c2 =', encrypt(m, n))
```

We are given a RSA moduli $n$ and encryption of flag $m$ under 2 different public exponent. Consider in Polynomial Ring Mod $n$, 2 polys:
$$f_{1}(m) = (m+noise_{1})^{e_{1}} - c_{1}$$
$$f_{2}(m) = (m+noise_{2})^{e_{2}} - c_{2}$$
Then $m$ is the common root of $f_{1}(m)$ and $f_{2}(m)$, which is also root of $g(m) = gcd(f_{1}(m), f_{2}(m))$. And because degree of $g(m)$ is small, we can easily calculate $m$.

In [1]:
from sage.all import *
from Crypto.Util.number import long_to_bytes

In [2]:
n = 99432613480939068351562426450736079548256649399824074125897023718511347184177748762719404609118999419018534660223144728190056735099787907299980625300234355248546050583144387977927309463501291854876892509630938617460690481497010165530214494306444999151252999850250583288798888278770654238342967653191171473013
e1, noise1, c1 = (9102, 42926763857648808452080305910241720054908809667539630194138718908195450776152522239791644645043372458139920503040529361726409749150633609223017012694569617522037971161448894459262110250322393703607247036022683527543284339763718964451482238661494995313111724858075982045508601405376724544741352142401794693054, 48276023282567629527195243870327301962656940680898728928903255577939008086657887592958073923577657060463242759606506812938152312008130198252498457257386413883443843887507528097024367788094619479032221547513746486475136282357337951126122694205225292004957793882304453164618423156810792171305978347365910972343)
e2, noise2, c2 = (2109, 51208643076502294588477225830948052764402322839126847164816681682357946991156728371602766970288519802146987999203830056494899211501025949997165558057140744445002699137286162872658309250096136525032077525028373299701055357023079519776378532002052890676446838318133048612893135724217301724754396467377231356425, 30644829500627448217295366947497931474953886995151259599263428251525601964766004111974074015504963773615137800165460045351514062357500899618814273135292073698096477339942069685331462828432407501524816375109607227118357281435280158409804228556720158131377342049528810546024786899763038442784789928604641662412)


In [3]:
def _gcd(a, b):
    while b != 0:
        a, b = b, a%b
    return a

In [4]:
R.<m> = PolynomialRing(Zmod(n))
f1 = (m + noise1)^e1 - c1
f2 = (m + noise2)^e2 - c2

g = _gcd(f1, f2)

a, b = g.coefficients()
x = -inverse_mod(ZZ(b), n) * a % n
print long_to_bytes(x)

TetCTF{1t_1s_4ll_4b0ut_GCD_0v3r_p0lyn0m14ls}
